# BioExplorer - Animated neurons
![](../../bioexplorer_neurons_banner.png)

### Connect to back-end

In [205]:
from bioexplorer import BioExplorer, CellAnimationParams

url = 'localhost:5000'
be = BioExplorer(url)

core = be.core_api()
status = be.reset_scene()

In [206]:
import seaborn as sns
def set_morphology_materials(model_id, palette_name, palette_size):
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    emissions = list()
    
    material_ids = be.get_material_ids(model_id)['ids']
    palette = sns.color_palette(palette_name, palette_size)

    for material_id in material_ids:
        mid = material_id % palette_size
        if mid == be.NEURON_MATERIAL_SYNAPSE:
            colors.append(palette[0])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.1)
            user_params.append(0.001)
            specular_exponents.append(5.0)
            emissions.append(0.0)
        elif mid == be.NEURON_MATERIAL_MITOCHONDRION:
            colors.append([1.0, 0.0, 1.0])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.5)
            user_params.append(1.0)
            specular_exponents.append(6.0)
            emissions.append(0.25)
        elif mid == be.NEURON_MATERIAL_NUCLEUS:
            colors.append([1.0, 1.0, 1.0])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(1.0)
            user_params.append(0.2)
            specular_exponents.append(30.0)
            emissions.append(0.0)
        elif mid == be.NEURON_MATERIAL_SOMA:
            colors.append(palette[0])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.2)
            user_params.append(0.01)
            specular_exponents.append(5.0)
            emissions.append(0.0)
        elif mid == be.NEURON_MATERIAL_MYELIN_STEATH:
            colors.append([0.4, 0.3, 0.5])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_DIFFUSE)
            glossinesses.append(1.0)
            user_params.append(1.0)
            specular_exponents.append(50.0)
            emissions.append(0.0)
        else:
            # Membrane
            colors.append(palette[0])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.1)
            user_params.append(0.1)
            specular_exponents.append(5.0)
            emissions.append(0.0)
            
        refraction_indices.append(1.0)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses,
        emissions=emissions
    )

### Load neurons

In [220]:
def add_neurons(seed, amplitude, frequency):
    assembly_name = 'Neurons'
    be.remove_assembly(assembly_name)
    neurons_assembly = be.add_assembly(assembly_name)

    neurons_model = be.add_neurons(
        assembly_name=assembly_name,
        population_name='o1',
        morphology_representation=be.MORPHOLOGY_REPRESENTATION_SECTION,
        morphology_color_scheme=be.MORPHOLOGY_COLOR_SCHEME_SECTION_TYPE,
        realism_level=be.MORPHOLOGY_REALISM_LEVEL_ALL,
        synapses_type=be.NEURON_SYNAPSES_NONE,
        load_somas=True, 
        load_axon=False, generate_varicosities=False,
        load_basal_dendrites=True, load_apical_dendrites=True,
        generate_externals=True,
        sql_node_filter='guid%10=1 AND SQRT(POW(x-345.0, 2.0) + POW(y-1357.0, 2.0) + POW(z-599.0, 2.0)) < 100',
        animation_params=CellAnimationParams(
            seed=seed, offset=100, amplitude=amplitude, frequency=frequency)
    )

In [212]:
status = core.set_camera(
    orientation=[0.0, 0.0, 0.0, 1.0],
    position=[320.76451674263274, 1301.2661854383591, 718.1424371666491],
    target=[320.76451674263274, 1301.2661854383591, 567.7575607299805],
)
status = core.set_renderer()

In [221]:
status = core.set_renderer(
    background_color=[0,0,0],
    current='advanced',subsampling=4, max_accum_frames=64)
params = core.AdvancedRendererParams()
params.gi_distance = 10.0
params.shadows = 1.0
params.soft_shadows = 0.0
params.epsilon_factor = 40.0
params.max_bounces = 10
params.show_background = True
params.exposure = 1.1
status = core.set_renderer_params(params)
status = core.set_renderer()

In [222]:
from bioexplorer import MovieMaker
mm = MovieMaker(be)
for i in range(0, 360):
    add_neurons(seed=i, amplitude=0.2, frequency=0.05)
    model_ids = be.get_model_ids()['ids']
    offset = model_ids[0]
    set_morphology_materials(offset, 'Reds', be.NB_MATERIALS_PER_MORPHOLOGY)
    status = core.set_renderer()
    mm.create_snapshot(
        renderer='advanced',
        path='/tmp', base_name='%05d' % i,
        size=[960, 540], samples_per_pixel=32)